# Analyze and filter DBTL1 recommendations
Filter the recommendations for DBTL1. `art_modeling_and_recommend_DBTL1` produced a dataframe consisting of every combination of 5 gRNAs, along with ART-predicted mean and std predictions for titer. Here, I'll filter and sort the recommendations based on a few criteria:
- changing exploration vs exploitation - Different recommendations will look better or worse based on whether we want to priorities exploration or exploitation
- Different requested numbers of 2-, 3-, 4-, and 5-gRNA recommendations - Experimental collaborators may want different numbers of each.
- Different levels of variety in gRNA - It's possible that ART identified two gRNAs (A and B, for instance) that it considers optimal, and every high-ranking recommendation for 3-gRNA strains will look like A + B + X. In this case, we may want to filter out some of these recommendations.